In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import init_chat_model
from langchain_chroma import Chroma
from langchain import hub
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

In [4]:
llm = init_chat_model("gpt-4o-mini", model_provider="openai")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vector_store = Chroma(
    collection_name="documents_db",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_docs_db",  
)

In [8]:
# Load and chunk contents of the blog
PDF_PATH = "data/rag.pdf"

pdf_loader = PyPDFLoader(PDF_PATH)
docs = pdf_loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(docs)

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

MAX_BATCH = 5461

# Index en lotes
for batch in chunks(all_splits, MAX_BATCH):
    vector_store.add_documents(documents=batch)

In [10]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

c:\Users\johan\Documents\code\ia_ps\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [11]:
response = graph.invoke({"question": "Qué significa RAG?"})
print(response["answer"])

RAG significa "Retrieval-Augmented Generation", que es un modelo que combina enfoques de generación de texto y recuperación de documentos. Este enfoque permite a los modelos generar respuestas más precisas y diversas al usar documentos relevantes como contexto adicional. RAG se destaca por lograr un rendimiento de vanguardia sin requerir entrenamiento de preprocesamiento costoso y especializado.


In [12]:
response = graph.invoke({"question": "Quienes crearon la tecnologia RAG?"})
print(response["answer"])

La tecnología RAG fue desarrollada por investigadores asociados con Hugging Face, en colaboración con conceptos y arquitecturas como el Dense Passage Retriever (DPR) y el modelo BART. RAG combina la generación flexible y el rendimiento de enfoques de recuperación, ofreciendo resultados competitivos sin la necesidad de preentrenamientos costosos. Se basa en técnicas previas de memoria no paramétrica pero fue optimizada para ser eficaz en tareas de generación y recuperación de información.


In [13]:
response = graph.invoke({"question": "En qué consiste la tecnologia RAG?"})
print(response["answer"])

La tecnología RAG (Retrieval-Augmented Generation) combina la generación de respuestas basada en modelos paramétricos y el uso de documentos recuperados como contexto adicional. Utiliza un recuperador para obtener pasajes relevantes en función de una consulta y luego genera respuestas considerando tanto la entrada como estos documentos recuperados. RAG muestra un rendimiento de vanguardia sin la necesidad de componentes adicionales como un lector extractivo o re-ranker.
